In [3]:
pip install python-Levenshtein


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ----------------------

In [5]:
# =============================================
# 📦 المكتبات المطلوبة
# =============================================
import numpy as np
import os
import joblib
import json
import re
from symspellpy import SymSpell, Verbosity
from gensim import corpora, models
import Levenshtein
from collections import Counter
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tabulate import tabulate

# تحميل البيانات
docs_dict = {}
with open("../data/msmarco_train/raw/raw_msmarco_train.json", "r", encoding="utf-8") as f:
    for line in f:
        try:
            doc = json.loads(line)
            docs_dict[str(doc["id"])] = doc["text"]
        except:
            continue


In [6]:
# =============================================
# 🔤 التصحيح الإملائي
# =============================================

# تهيئة SymSpell
sym_spell = SymSpell()
dictionary_path = "../data/frequency_dictionary_en_82_765.txt"
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

def correct_spelling(query):
    """تصحيح الأخطاء الإملائية في الاستعلام"""
    words = query.lower().split()
    corrected_words = []
    
    for word in words:
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST)
        if suggestions:
            corrected_words.append(suggestions[0].term)
        else:
            corrected_words.append(word)
    
    corrected_query = " ".join(corrected_words)
    return corrected_query

# اختبار التصحيح الإملائي
test_queries = [
    "plaing football",
    "computr science",
    "artifical inteligence"
]

print("📝 نتائج التصحيح الإملائي:")
for query in test_queries:
    corrected = correct_spelling(query)
    print(f"الاستعلام الأصلي: {query}")
    print(f"التصحيح: {corrected}")
    print("-" * 50)


2025-06-30 22:17:29,310: E symspellpy.symspellpy] Dictionary file not found at ..\data\frequency_dictionary_en_82_765.txt.


📝 نتائج التصحيح الإملائي:
الاستعلام الأصلي: plaing football
التصحيح: plaing football
--------------------------------------------------
الاستعلام الأصلي: computr science
التصحيح: computr science
--------------------------------------------------
الاستعلام الأصلي: artifical inteligence
التصحيح: artifical inteligence
--------------------------------------------------


In [7]:
# =============================================
# 📚 اكتشاف المواضيع
# =============================================

# تجهيز البيانات لنموذج LDA
texts = [doc.lower().split() for doc in list(docs_dict.values())[:1000]]  # نستخدم عينة للسرعة
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# تدريب نموذج LDA
num_topics = 10
lda_model = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_topics,
    random_state=42,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha='auto'
)

def detect_topics(query, top_n=3):
    """اكتشاف المواضيع في الاستعلام"""
    # تحويل الاستعلام إلى Bag of Words
    query_bow = dictionary.doc2bow(query.lower().split())
    
    # الحصول على توزيع المواضيع
    topic_distribution = lda_model.get_document_topics(query_bow, minimum_probability=0)
    
    # ترتيب المواضيع حسب الاحتمالية
    sorted_topics = sorted(topic_distribution, key=lambda x: x[1], reverse=True)
    
    # إرجاع أهم المواضيع
    results = []
    for topic_id, prob in sorted_topics[:top_n]:
        topic_terms = lda_model.show_topic(topic_id, topn=5)
        terms = [term for term, _ in topic_terms]
        results.append({
            'topic_id': topic_id,
            'probability': prob,
            'top_terms': terms
        })
    return results

# اختبار اكتشاف المواضيع
test_queries = [
    "machine learning algorithms",
    "climate change effects",
    "healthy diet and nutrition"
]

print("🎯 نتائج اكتشاف المواضيع:")
for query in test_queries:
    print(f"\nالاستعلام: {query}")
    topics = detect_topics(query)
    for i, topic in enumerate(topics, 1):
        print(f"الموضوع {i}:")
        print(f"- الاحتمالية: {topic['probability']:.4f}")
        print(f"- المصطلحات الرئيسية: {', '.join(topic['top_terms'])}")
    print("-" * 50)


🎯 نتائج اكتشاف المواضيع:

الاستعلام: machine learning algorithms
الموضوع 1:
- الاحتمالية: 0.5468
- المصطلحات الرئيسية: the, of, and, in, a
الموضوع 2:
- الاحتمالية: 0.2219
- المصطلحات الرئيسية: they, years, i, largest, are
الموضوع 3:
- الاحتمالية: 0.1286
- المصطلحات الرئيسية: you, or, your, if, 1
--------------------------------------------------

الاستعلام: climate change effects
الموضوع 1:
- الاحتمالية: 0.4793
- المصطلحات الرئيسية: the, of, and, in, a
الموضوع 2:
- الاحتمالية: 0.1939
- المصطلحات الرئيسية: they, years, i, largest, are
الموضوع 3:
- الاحتمالية: 0.1569
- المصطلحات الرئيسية: population, states, much, used, show
--------------------------------------------------

الاستعلام: healthy diet and nutrition
الموضوع 1:
- الاحتمالية: 0.5358
- المصطلحات الرئيسية: the, of, and, in, a
الموضوع 2:
- الاحتمالية: 0.2115
- المصطلحات الرئيسية: you, or, your, if, 1
الموضوع 3:
- الاحتمالية: 0.1225
- المصطلحات الرئيسية: field, periodic, group, annual, big
----------------------------------------

In [8]:
# =============================================
# 💡 اقتراحات الاستعلام
# =============================================

# بناء قاموس المصطلحات المرتبطة
def build_related_terms_dict(docs_sample=1000):
    """بناء قاموس المصطلحات المرتبطة من مجموعة من الوثائق"""
    related_terms = {}
    term_freq = Counter()
    
    # استخدام عينة من الوثائق للسرعة
    sample_docs = list(docs_dict.values())[:docs_sample]
    
    # حساب تكرار المصطلحات وبناء العلاقات
    for doc in sample_docs:
        terms = doc.lower().split()
        # تحديث تكرارات المصطلحات
        term_freq.update(terms)
        
        # بناء العلاقات بين المصطلحات
        for i, term in enumerate(terms):
            window = terms[max(0, i-3):min(len(terms), i+4)]  # نافذة سياق 3 كلمات قبل وبعد
            for other_term in window:
                if term != other_term:
                    related_terms.setdefault(term, Counter()).update([other_term])
    
    return related_terms, term_freq

def suggest_queries(query, related_terms, term_freq, max_suggestions=5):
    """توليد اقتراحات للاستعلام"""
    query_terms = query.lower().split()
    suggestions = []
    
    # توليد اقتراحات بناءً على المصطلحات المرتبطة
    for term in query_terms:
        if term in related_terms:
            # الحصول على أكثر المصطلحات ارتباطاً
            related = related_terms[term].most_common(3)
            for rel_term, count in related:
                # تجنب المصطلحات النادرة جداً
                if term_freq[rel_term] > 5:
                    # إنشاء استعلام جديد باستبدال المصطلح الحالي
                    new_terms = query_terms.copy()
                    new_terms[new_terms.index(term)] = rel_term
                    suggestions.append(" ".join(new_terms))
    
    # إضافة مصطلحات إضافية للاستعلام
    for term in query_terms:
        if term in related_terms:
            for rel_term, count in related_terms[term].most_common(2):
                if term_freq[rel_term] > 5:
                    suggestions.append(f"{query} {rel_term}")
    
    # إزالة التكرارات وترتيب الاقتراحات
    suggestions = list(set(suggestions))
    suggestions = sorted(suggestions, key=len)[:max_suggestions]
    return suggestions

# بناء قاموس المصطلحات المرتبطة
print("🔄 جاري بناء قاموس المصطلحات المرتبطة...")
related_terms, term_freq = build_related_terms_dict()

# اختبار اقتراحات الاستعلام
test_queries = [
    "machine learning",
    "data science",
    "artificial intelligence"
]

print("\n💡 نتائج اقتراحات الاستعلام:")
for query in test_queries:
    print(f"\nالاستعلام الأصلي: {query}")
    suggestions = suggest_queries(query, related_terms, term_freq)
    print("الاقتراحات:")
    for i, suggestion in enumerate(suggestions, 1):
        print(f"{i}. {suggestion}")
    print("-" * 50)


🔄 جاري بناء قاموس المصطلحات المرتبطة...

💡 نتائج اقتراحات الاستعلام:

الاستعلام الأصلي: machine learning
الاقتراحات:
1. a learning
2. machine the
3. machine going
4. machine through
5. running learning
--------------------------------------------------

الاستعلام الأصلي: data science
الاقتراحات:
1. for science
2. the science
3. census science
4. data science the
5. data science census
--------------------------------------------------

الاستعلام الأصلي: artificial intelligence
الاقتراحات:
1. is intelligence
2. the intelligence
3. language intelligence
4. artificial intelligence is
5. artificial intelligence the
--------------------------------------------------
